<a href="https://colab.research.google.com/github/aiscience-22/UA_War/blob/machine_learning_0.01/Postgres_Reading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Install required tensorflow-io packages
try:
  %tensorflow_version 2.x
except Exception:
  pass

!pip install tensorflow-io

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `databootcamp2022` for username `uatweets`
#!sudo -u postgres psql -U postgres -c "ALTER USER uatweets PASSWORD 'databootcamp2022';"


 * Starting PostgreSQL 10 database server
   ...done.
ERROR:  role "uatweets" does not exist


In [13]:
# Setup necessary environmental variables
%env TFIO_DEMO_DATABASE_NAME=Tweets
%env TFIO_DEMO_DATABASE_HOST=tweets.cnzbbvrrhst7.us-west-1.rds.amazonaws.com
%env TFIO_DEMO_DATABASE_PORT=5432
%env TFIO_DEMO_DATABASE_USER=uatweets
%env TFIO_DEMO_DATABASE_PASS=databootcamp2022

env: TFIO_DEMO_DATABASE_NAME=Tweets
env: TFIO_DEMO_DATABASE_HOST=tweets.cnzbbvrrhst7.us-west-1.rds.amazonaws.com
env: TFIO_DEMO_DATABASE_PORT=5432
env: TFIO_DEMO_DATABASE_USER=uatweets
env: TFIO_DEMO_DATABASE_PASS=databootcamp2022


In [4]:
# The complete commands to create the table in database and populate the data

!curl -s -OL https://github.com/tensorflow/io/raw/master/docs/tutorials/postgresql/AirQualityUCI.sql

 !PGPASSWORD=$databootcamp2022 psql -q -h $tweets.cnzbbvrrhst7.us-west-1.rds.amazonaws.com -p $5432 -U $uatweets -d $Tweets -f AirQualityUCI.sql

/bin/bash: psql: command not found


# Create Dataset from PostgreSQL server and use it in TensorFlow

Create a Dataset from PostgreSQL server is as easy as calling tfio.experimental.IODataset.from_sql with query and endpoint arguments. The query is the SQL query for select columns in tables and the endpoint argument is the address and database name:

tweets.cnzbbvrrhst7.us-west-1.rds.amazonaws.com  
Port 5432  
Username: uatweets  
password: databootcamp2022  

In [14]:
!pip install tensorflow_io
import os
import tensorflow_io as tfio

endpoint="postgresql://{uatweets}:{databootcamp2022}@{tweets.cnzbbvrrhst7.us-west-1.rds.amazonaws.com}?port={5432}&dbname={Tweets}".format(
    os.environ['TFIO_DEMO_DATABASE_USER'],
    os.environ['TFIO_DEMO_DATABASE_USER'],
    os.environ['TFIO_DEMO_DATABASE_HOST'],
    os.environ['TFIO_DEMO_DATABASE_PORT'],
    os.environ['TFIO_DEMO_DATABASE_NAME'],
)


dataset = tfio.experimental.IODataset.from_sql(
    query="SELECT * FROM joined;",
    endpoint=endpoint)

print(dataset.element_spec)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


KeyError: ignored

The output of dataset.element_spec above, the element of the created Dataset is a python dict object with column names of the database table as keys. It is quite convenient to apply further operations. For example, you could select both nox and no2 field of the Dataset, and calculate the difference:

In [ ]:
dataset = tfio.experimental.IODataset.from_sql(
    query="SELECT nox, no2 FROM AirQualityUCI;",
    endpoint=endpoint)

dataset = dataset.map(lambda e: (e['nox'] - e['no2']))

# check only the first 20 record
dataset = dataset.take(20)

print("NOx - NO2:")
for difference in dataset:
  print(difference.numpy())